# **Train data on the Cifar10 dataset using Traffic Sign Classifier**

In [1]:
%tensorflow_version 1.x
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# y_train.shape is 2d, (50000, 1). While Keras is smart enough to handle this
# it's a good idea to flatten the array.
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify = y_train)

In [0]:
import tensorflow as tf

EPOCHS = 10
BATCH_SIZE = 128

In [0]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    F_W1 = tf.Variable(tf.truncated_normal([5, 5, 3, 6], mean=mu, stddev=sigma))
    F_b1 = tf.Variable(tf.zeros(6))   
    strides1 = [1, 1, 1, 1]
    padding = 'VALID'
    Conv1 = tf.nn.conv2d(x, F_W1, strides1, padding) + F_b1

    # TODO: Activation.
    Conv1 = tf.nn.relu(Conv1)

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    Conv1 = tf.nn.max_pool(
        Conv1,
        ksize=[1, 2, 2, 1],
        strides=[1, 2, 2, 1],
        padding='VALID')
    Conv1 = tf.nn.dropout(Conv1, keep_prob)
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    F_W2 = tf.Variable(tf.truncated_normal([5, 5, 6, 16], mean=mu, stddev=sigma))
    F_b2 = tf.Variable(tf.zeros(16))   
    strides2 = [1, 1, 1, 1]
    padding = 'VALID'
    Conv2 = tf.nn.conv2d(Conv1, F_W2, strides2, padding) + F_b2
    
    # TODO: Activation.
    Conv2 = tf.nn.relu(Conv2)

    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    Conv2 = tf.nn.max_pool(
        Conv2,
        ksize=[1, 2, 2, 1],
        strides=[1, 2, 2, 1],
        padding='VALID')
    Conv2 = tf.nn.dropout(Conv2, keep_prob)

    # TODO: Flatten. Input = 5x5x16. Output = 400.
    flat = flatten(Conv2)
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    F_W3 = tf.Variable(tf.truncated_normal([400, 120], mean=mu, stddev=sigma))
    F_b3 = tf.Variable(tf.zeros(120))
    FC3 = tf.matmul(flat, F_W3) + F_b3

    # TODO: Activation.
    FC3 = tf.nn.relu(FC3)
    FC3 = tf.nn.dropout(FC3, keep_prob)

    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    F_W4 = tf.Variable(tf.truncated_normal([120, 84], mean=mu, stddev=sigma))
    F_b4 = tf.Variable(tf.zeros(84))
    FC4 = tf.matmul(FC3, F_W4) + F_b4
    
    # TODO: Activation.
    FC4 = tf.nn.relu(FC4)
    FC4 = tf.nn.dropout(FC4, keep_prob)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 43.
    F_W5 = tf.Variable(tf.truncated_normal([84, 43], mean=mu, stddev=sigma))
    F_b5 = tf.Variable(tf.zeros(43))
    logits = tf.matmul(FC4, F_W5) + F_b5
    
    return logits

In [0]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

# Placeholders
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)

In [6]:
# Training Pipeline

rate = 0.001

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# Model Evaluation
prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(prediction, tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [8]:
# Train the model
from sklearn.utils import shuffle
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.9})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './Cifar10')
    print("Model saved")

Training...

EPOCH 1 ...
Validation Accuracy = 0.249

EPOCH 2 ...
Validation Accuracy = 0.324

EPOCH 3 ...
Validation Accuracy = 0.381

EPOCH 4 ...
Validation Accuracy = 0.408

EPOCH 5 ...
Validation Accuracy = 0.421

EPOCH 6 ...
Validation Accuracy = 0.438

EPOCH 7 ...
Validation Accuracy = 0.451

EPOCH 8 ...
Validation Accuracy = 0.450

EPOCH 9 ...
Validation Accuracy = 0.463

EPOCH 10 ...
Validation Accuracy = 0.475

Model saved
